**개요 : 영윤의 가장 최근 데이터를 이용해서 재료, 설명, 레시피에 따른 추천 음식을 출력하고 비교해봄**

In [121]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [122]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MachineLearningPractice/Colab-Notebooks/FarmKit/foodfile/220721_정영윤_ver3 (영문, 병합 번역).csv')

In [123]:
df.head()

,id,카테고리명,분류기준,식품명,설명,ingre,recipe
0,1,양식,그라탕류,라따뚜이그라탕,"It is a gratin using ratatouille, a traditiona...","Tomato, eggplant, zucchini, penne, tomato sauc...","Prepare tomatoes, eggplant, and zucchini by cu..."
1,2,간식,토스트류,햄치즈토스트,Sliced ​​ham and cheese are placed on bread an...,"White bread, sliced ​​ham, cheddar cheese, can...",Prepare the cabbage and carrots by slicing the...
2,3,밑반찬,건어물류,마른미역자반,It is a side dish of dried seaweed fried in oi...,"Dried seaweed, perilla oil, cooking oil, sugar...",Heat perilla oil and cooking oil in a pan. Whe...
3,4,한식,찌개류,돼지고기된장찌개,It is a stew made by slicing pork and putting ...,"Pork leg meat, green onion, onion, zucchini, r...",Cut the onion and pumpkin into appropriate siz...
4,5,간식,빵류,고구마치즈빵,A sweet-tasting bread topped with boiled sweet...,"Steamed sweet potato, mayonnaise, honey, mozza...","Mix flour, milk, melted butter and salt in a b..."


In [124]:
df.shape

(2381, 7)

In [125]:
df = df.dropna()    # 결손 데이터
df.shape

(2343, 7)

In [126]:
df.reset_index(inplace=True)

**레시피에 대한 피처 벡터화**

In [127]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')     # 크게 의미없는 단어들을 제외해준다.
tfidf_matrix = tfidf.fit_transform(df['recipe'])
tfidf_matrix.shape

(2343, 3462)

**설명에 대한 피처 벡터화**

In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf2 = TfidfVectorizer(stop_words='english')     # 크게 의미없는 단어들을 제외해준다.
tfidf_matrix2 = tfidf2.fit_transform(df['설명'])
tfidf_matrix2.shape

(2343, 2023)

In [129]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim_recipe = linear_kernel(tfidf_matrix, tfidf_matrix)     # 코사인 유사도 계산
cosine_sim_recipe

array([[1.        , 0.06774809, 0.01612975, ..., 0.04568099, 0.02942222,
        0.1179793 ],
       [0.06774809, 1.        , 0.06186918, ..., 0.03076956, 0.04576709,
        0.15774536],
       [0.01612975, 0.06186918, 1.        , ..., 0.00779477, 0.06149775,
        0.11839264],
       ...,
       [0.04568099, 0.03076956, 0.00779477, ..., 1.        , 0.11404926,
        0.02412688],
       [0.02942222, 0.04576709, 0.06149775, ..., 0.11404926, 1.        ,
        0.04042471],
       [0.1179793 , 0.15774536, 0.11839264, ..., 0.02412688, 0.04042471,
        1.        ]])

In [130]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim_sum = linear_kernel(tfidf_matrix2, tfidf_matrix2)     # 코사인 유사도 계산
cosine_sim_sum

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.07437211,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.180337  ,
        0.        ],
       [0.        , 0.07437211, 0.        , ..., 0.180337  , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [131]:
indices = pd.Series(df.index, index=df['식품명']).drop_duplicates()    # 중복 제거
indices

식품명
라따뚜이그라탕        0
햄치즈토스트         1
마른미역자반         2
돼지고기된장찌개       3
고구마치즈빵         4
            ... 
홍루이젠        2338
불고기우동샐러드    2339
바나나경단       2340
바나나로띠       2341
닭가슴살채소덮밥    2342
Length: 2343, dtype: int64

In [132]:
# 음식명을 입력받으면 코사인 유사도를 통해서 가장 유사도가 높은 상위 10개의 음식 목록 반환
def get_recommendations(title, cosine_sim):

  # 음식명을 통해서 전체 데이터 기준 그 음식의 index 값을 얻기
  idx = indices[title]

  # 코사인 유사도 매트릭스 (cosine_sim) 에서 idx 에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
  sim_scores = list(enumerate(cosine_sim[idx]))

  # 코사인 유사도 기준으로 내림차순 정렬
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # 자기 자신을 제외한 10개의 추천 음식을 슬라이싱
  sim_scores = sim_scores[1:11]

  # 추천 음식목록 10개의 인덱스 정보 추출
  movie_indices = [i[0] for i in sim_scores]

  # 인덱스 정보를 통해 음식명 추출
  return df['식품명'].iloc[movie_indices], sim_scores

In [133]:
get_recommendations('햄치즈토스트', cosine_sim_recipe)

(1750        양배추샌드위치
 1503        달걀채소토스트
 1970         모닝샌드위치
 1745     참치마요네즈샌드위치
 2049       베이컨치즈토스트
 109     몬테크리스토 샌드위치
 1452         프렌치토스트
 62          식빵 누네띠네
 44            밤잼토스트
 143         치즈갈릭브레드
 Name: 식품명, dtype: object,
 [(1750, 0.5583542622159365),
  (1503, 0.4911837960293013),
  (1970, 0.4604682863863688),
  (1745, 0.44288797985746137),
  (2049, 0.42928758078062074),
  (109, 0.42407542764146977),
  (1452, 0.41283524673067334),
  (62, 0.3869127569312986),
  (44, 0.38536212067669906),
  (143, 0.356698783749324)])

In [134]:
get_recommendations('햄치즈토스트', cosine_sim_sum)

(2338           홍루이젠
 109     몬테크리스토 샌드위치
 1312         몬테크리스토
 1016        레어치즈케이크
 1520          허니브레드
 982           식빵이자냐
 1735        새우그라탕덮밥
 1477          크림통식빵
 286           닭날개구이
 1219            허니윙
 Name: 식품명, dtype: object,
 [(2338, 0.3752049712098212),
  (109, 0.3616530284574012),
  (1312, 0.3616530284574012),
  (1016, 0.32955591594536254),
  (1520, 0.3295012345221366),
  (982, 0.3016089280262119),
  (1735, 0.2975943803201359),
  (1477, 0.2904661067483425),
  (286, 0.2658413398511886),
  (1219, 0.2658413398511886)])

In [135]:
get_recommendations('고구마치즈빵', cosine_sim_recipe)

(46      제티체크쿠키&제티설기
 428            감자피자
 173             소금빵
 1095            공갈빵
 1506       고구마베이컨말이
 2030          참깨크래커
 2328           시나몬롤
 624             모닝빵
 1442         감자찹쌀구이
 1261           사과파이
 Name: 식품명, dtype: object,
 [(46, 0.42383931821332677),
  (428, 0.39070380610206834),
  (173, 0.38218787944005767),
  (1095, 0.38159854501709367),
  (1506, 0.3802673339818307),
  (2030, 0.376664103873594),
  (2328, 0.3674688508088682),
  (624, 0.3441799598786557),
  (1442, 0.3441098617395143),
  (1261, 0.3217325824008149)])

In [136]:
get_recommendations('고구마치즈빵', cosine_sim_sum)

(1791          더블치즈토스트
 1867            치즈마늘빵
 2049         베이컨치즈토스트
 104              프로틴빵
 750               구름빵
 824     리얼고구마빵&리얼 감자빵
 2085            하이토스트
 1477            크림통식빵
 2022           고구마치즈볼
 109       몬테크리스토 샌드위치
 Name: 식품명, dtype: object,
 [(1791, 0.40598169259222044),
  (1867, 0.39094233450721444),
  (2049, 0.33919580716788794),
  (104, 0.3329315020739334),
  (750, 0.3261814291513759),
  (824, 0.32121583096339595),
  (2085, 0.3194057291094262),
  (1477, 0.31133548600682365),
  (2022, 0.30385697548965707),
  (109, 0.30137622075849185)])

In [137]:
get_recommendations('닭가슴살채소덮밥', cosine_sim_recipe)

(1396       닭가슴살채소볶음
 98            오트밀카레
 635       후루룩닭고기쌀국수
 102           순두부인헬
 1576         닭가슴살쌈밥
 232           오트밀닭죽
 1768      닭가슴살오픈토스트
 1604    통바게트양배추샌드위치
 1851         감자칩카나페
 2061        치킨김치볶음밥
 Name: 식품명, dtype: object,
 [(1396, 0.6626204366720178),
  (98, 0.5263661050093844),
  (635, 0.5202898416673754),
  (102, 0.517842884551279),
  (1576, 0.48111493877338435),
  (232, 0.47782704150449),
  (1768, 0.44837424371438067),
  (1604, 0.442867338533518),
  (1851, 0.4422963317369643),
  (2061, 0.429931735159838)])

In [138]:
get_recommendations('닭가슴살채소덮밥', cosine_sim_sum)

(395          두부김밥
 1218        된장리조또
 563         김치볶음밥
 533           두부밥
 205      곤약새우버섯솥밥
 2098         오야꼬동
 497           곤약밥
 635     후루룩닭고기쌀국수
 156          꼬막덮밥
 1896     데리야끼치킨덮밥
 Name: 식품명, dtype: object,
 [(395, 0.7236781988477023),
  (1218, 0.6950397336298604),
  (563, 0.6064140538328923),
  (533, 0.5824694470851853),
  (205, 0.47735276378119806),
  (2098, 0.35768661212233455),
  (497, 0.3466546733697208),
  (635, 0.3405143242450578),
  (156, 0.3323187683274805),
  (1896, 0.3178636576716187)])

**재료를 기준으로 음식을 추천해줌**

In [139]:
df['ingre']

0       Tomato, eggplant, zucchini, penne, tomato sauc...
1       White bread, sliced ​​ham, cheddar cheese, can...
2       Dried seaweed, perilla oil, cooking oil, sugar...
3       Pork leg meat, green onion, onion, zucchini, r...
4       Steamed sweet potato, mayonnaise, honey, mozza...
                              ...                        
2338    Bread, egg, sliced ​​ham, sliced ​​cheese, may...
2339    Beef for bulgogi, udon noodles, lettuce, soy s...
2340    Banana, glutinous rice flour (dry), salt, cast...
2341    Banana, tortilla, egg, butter, condensed milk,...
2342    Boiled chicken breast, cabbage, onion, carrot,...
Name: ingre, Length: 2343, dtype: object

In [140]:
type(df['ingre'][0])

str

In [141]:
def ingredient(x):
  x = x.split(',')[:4]
  return ' '.join(x)

In [142]:
# df['ingre'] = df['ingre'].apply(lambda x : x.split(','))
# df['ingre'] = df['ingre'].apply(lambda x : x[:5])
# df['ingre'] = df['ingre'].apply(lambda x : ' '.join(x))
df['ingre'] = df['ingre'].apply(ingredient)
df['ingre']

0                       Tomato  eggplant  zucchini  penne
1       White bread  sliced ​​ham  cheddar cheese  can...
2          Dried seaweed  perilla oil  cooking oil  sugar
3             Pork leg meat  green onion  onion  zucchini
4       Steamed sweet potato  mayonnaise  honey  mozza...
                              ...                        
2338            Bread  egg  sliced ​​ham  sliced ​​cheese
2339    Beef for bulgogi  udon noodles  lettuce  soy s...
2340    Banana  glutinous rice flour (dry)  salt  cast...
2341                        Banana  tortilla  egg  butter
2342        Boiled chicken breast  cabbage  onion  carrot
Name: ingre, Length: 2343, dtype: object

In [143]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
count_matrix3 = count.fit_transform(df['ingre'])
count_matrix3

<2343x1023 sparse matrix of type '<class 'numpy.int64'>'
	with 14521 stored elements in Compressed Sparse Row format>

In [144]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim_ingre = cosine_similarity(count_matrix3, count_matrix3)
cosine_sim_ingre

array([[1.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 1.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 1.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 1.       , 0.1767767,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.1767767, 1.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ]])

In [145]:
get_recommendations('햄치즈토스트', cosine_sim_ingre)

(2049       베이컨치즈토스트
 1312         몬테크리스토
 272        오믈렛컵&피자컵
 477           구름토스트
 1874          크로크마담
 447          양파햄치즈빵
 2338           홍루이젠
 1839         치즈폭탄피자
 109     몬테크리스토 샌드위치
 219           피자토스트
 Name: 식품명, dtype: object,
 [(2049, 0.686406472983644),
  (1312, 0.6708203932499369),
  (272, 0.6681531047810608),
  (477, 0.6681531047810608),
  (1874, 0.6249999999999999),
  (447, 0.6249999999999998),
  (2338, 0.6249999999999998),
  (1839, 0.5590169943749475),
  (109, 0.5345224838248487),
  (219, 0.4999999999999999)])

In [146]:
get_recommendations('고구마치즈빵', cosine_sim_ingre)

(1591      단호박치즈구이
 2022       고구마치즈볼
 802         고구마경단
 831      아코디언꿀고구마
 1401      오지치즈후라이
 695        고구마핫도그
 1878       고구마말랭이
 1506     고구마베이컨말이
 1532    훈제오리단호박구이
 1636     인절미고구마맛탕
 Name: 식품명, dtype: object,
 [(1591, 0.5976143046671968),
  (2022, 0.5976143046671968),
  (802, 0.50709255283711),
  (831, 0.50709255283711),
  (1401, 0.5050762722761053),
  (695, 0.4629100498862757),
  (1878, 0.4364357804719848),
  (1506, 0.4285714285714285),
  (1532, 0.4285714285714285),
  (1636, 0.4285714285714285)])

In [147]:
get_recommendations('닭가슴살채소덮밥', cosine_sim_ingre)

(98         오트밀카레
 1396    닭가슴살채소볶음
 232        오트밀닭죽
 364       치즈간장찜닭
 1249         코코뱅
 1263        카레찜닭
 1541        마라찜닭
 861         코울슬로
 1093     로스트치즈치킨
 1941      치킨크림스튜
 Name: 식품명, dtype: object,
 [(98, 0.7302967433402215),
  (1396, 0.7302967433402215),
  (232, 0.6666666666666669),
  (364, 0.6123724356957946),
  (1249, 0.6123724356957946),
  (1263, 0.6123724356957946),
  (1541, 0.6123724356957946),
  (861, 0.5477225575051662),
  (1093, 0.5477225575051662),
  (1941, 0.5477225575051662)])

In [148]:
print('닭가슴살채소덮밥 ingre : ' + df.iloc[2342]['ingre'] + '\n오트밀카레 ingre : '+ df.iloc[98]['ingre'] + '\n닭가슴살채소볶음 ingre : ' + df.iloc[1396]['ingre'])

닭가슴살채소덮밥 ingre : Boiled chicken breast  cabbage  onion  carrot
오트밀카레 ingre : Oatmeal  chicken breast  onion  carrot
닭가슴살채소볶음 ingre : Chicken breast  zucchini  carrot  onion
